In [1]:
%load_ext autoreload
%autoreload 2
import torch
import numpy as np
from datasets import TripletAudio
from networks import AnchorNet, EmbeddingNet, TripletNet
from losses import TripletLoss
import torch.optim as optim
from recall import Recall
from torch.optim import lr_scheduler
from trainer import fit
import itertools
from tensorboardX import SummaryWriter

In [2]:
#define hyperparams
K, MAX_CLOSE_NEG, P_STRONG_NEG = 5, 15, 1
BATCH_SIZE = 1
INPUT_D, OUTPUT_D = 192, 128
MARGIN, N_EPOCHS, LOG_INT, N_RECALL_CAND = 0.5, 20, 100, 25
RHO, BETA = 5, 1

In [3]:
cuda = torch.cuda.is_available()
#define datasets
triplet_train_dataset = TripletAudio(True, K, MAX_CLOSE_NEG, P_STRONG_NEG)
triplet_test_dataset = TripletAudio(False, K, MAX_CLOSE_NEG, P_STRONG_NEG)
triplet_train_loader = torch.utils.data.DataLoader(triplet_train_dataset, batch_size=BATCH_SIZE, shuffle=True)
triplet_test_loader = torch.utils.data.DataLoader(triplet_test_dataset, batch_size=BATCH_SIZE, shuffle=False)
#define model 
anchor_net = AnchorNet(triplet_train_dataset.get_dataset(), INPUT_D, OUTPUT_D)
embedding_net = EmbeddingNet(anchor_net)
model = TripletNet(embedding_net)
if cuda:
    print("GPU Enabled")
    model.cuda()
loss_fn = TripletLoss(MARGIN, RHO, BETA)
optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-4)
scheduler = lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.1, last_epoch=-1)
#run the model
train_loss, val_loss = fit(triplet_train_loader, triplet_test_loader, model, loss_fn, optimizer, scheduler, N_EPOCHS, cuda, LOG_INT)

initialising model biases
done
GPU Enabled


/opt/anaconda3/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:82: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule.See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


RuntimeError: CUDA error: invalid configuration argument